# Homework: Vector Search

In [8]:
import numpy as np
from fastembed import TextEmbedding

# Q1. Embedding the query

In [27]:
# R:. -0.11
query = 'I just discovered the course. Can I join now?'
model_handle = "jinaai/jina-embeddings-v2-small-en"

# Embedding
embedding_model = TextEmbedding(model_name=model_handle)

embeddings = embedding_model.embed(query)
vector = list(embeddings)[0] 
print(vector.min())


-0.11726373885183883


# Q2. Cosine similarity with another vector

In [ ]:
# R:. 0.90
doc = 'Can I still join the course after the start date?'

embed_doc = embedding_model.embed(doc)
vector_doc = list(embed_doc)[0]

#Similarity
vector.dot(vector_doc)

np.float64(0.9008528895674548)

# Q3. Ranking by cosine

In [ ]:
# r:.1
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [64]:
V=[]
for doc in documents: 
    embeded = embedding_model.embed(doc['text'])
    v = list(embeded)[0]
    sm = v.dot(vector)
    V.append(sm)

In [ ]:
np.argmax(V)

np.int64(1)

# Q4. Ranking by cosine, version two

In [ ]:
#R:. 0 
V=[]
for doc in documents: 
    full_text = doc['question'] + ' ' + doc['text']
    embeded = embedding_model.embed(full_text)
    v = list(embeded)[0]
    sm = v.dot(vector)
    V.append(sm)

In [74]:
V

[np.float64(0.8514543236908068),
 np.float64(0.8436594159113068),
 np.float64(0.8408287048502558),
 np.float64(0.7755157969663908),
 np.float64(0.8086007795043938)]

# Q5. Selecting the embedding model

In [ ]:
# R:.384
dims = {}
for model in TextEmbedding.list_supported_models():
    dims[model['model']] = model['dim']

In [78]:
sorted(dims.items(), key=lambda item: item[1])

[('BAAI/bge-small-en', 384),
 ('BAAI/bge-small-en-v1.5', 384),
 ('snowflake/snowflake-arctic-embed-xs', 384),
 ('snowflake/snowflake-arctic-embed-s', 384),
 ('sentence-transformers/all-MiniLM-L6-v2', 384),
 ('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', 384),
 ('BAAI/bge-small-zh-v1.5', 512),
 ('Qdrant/clip-ViT-B-32-text', 512),
 ('jinaai/jina-embeddings-v2-small-en', 512),
 ('BAAI/bge-base-en', 768),
 ('BAAI/bge-base-en-v1.5', 768),
 ('snowflake/snowflake-arctic-embed-m', 768),
 ('snowflake/snowflake-arctic-embed-m-long', 768),
 ('jinaai/jina-clip-v1', 768),
 ('jinaai/jina-embeddings-v2-base-en', 768),
 ('jinaai/jina-embeddings-v2-base-de', 768),
 ('jinaai/jina-embeddings-v2-base-code', 768),
 ('jinaai/jina-embeddings-v2-base-zh', 768),
 ('jinaai/jina-embeddings-v2-base-es', 768),
 ('thenlper/gte-base', 768),
 ('nomic-ai/nomic-embed-text-v1.5', 768),
 ('nomic-ai/nomic-embed-text-v1.5-Q', 768),
 ('nomic-ai/nomic-embed-text-v1', 768),
 ('sentence-transformers/paraphrase

# Q6. Indexing with qdrant (2 points)

In [85]:
from qdrant_client import QdrantClient, models

COLLECTION_NAME = "homework_q6"
EMBEDDING_DIMENSIONALITY = 384
MODEL_HANDLE = "BAAI/bge-small-en"

In [86]:
# BAAI/bge-small-en
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [87]:
client = QdrantClient("http://localhost:6333")

# Creating the collection
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

# Create a index
client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="course",
    field_schema="keyword"
)

points = []

for i, doc in enumerate(documents):

        text = doc['question'] + ' ' + doc['text']
        vector= models.Document(text=text, model=MODEL_HANDLE)
        point = models.PointStruct(
            id=i,
            vector=vector,
            payload=doc
        )
        points.append(point)


# uspsert 
client.upsert(
    collection_name=COLLECTION_NAME,
    points=points
)



Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [93]:
def vector_search(query, limit):
    
    points= client.query_points(
        collection_name=COLLECTION_NAME,
        query=models.Document(
            text=query,
            model=MODEL_HANDLE
        ),
        limit=limit,
        with_payload=True)

    results=[]

    for point in points.points:
        results.append(point.score)

    return results 


In [94]:
vector_search(query=query, limit=1)

[0.8703172]